In [1]:
from boson_data_lib import *
import numpy as np
import scipy.linalg as sl
import os
import pandas as pd
import glob

In [2]:
#parent_dir = r"C:\Users\Zakhar\Documents\GitHub\POP23\"
parent_dir = "/home/zah/POP23/"
data_dir = parent_dir + "DATA/"
tests_dir = parent_dir + "TESTS/"

In [3]:
best_init_rho =  np.array([[ 0.96693975+0.j        , -0.17832301+0.00560344j],
                               [-0.17832301-0.00560344j,  0.03306025+0.j        ]])

fid_dists = fidelity_distances(data_dir, best_init_rho);

In [4]:
def read_as_dataframe_fits(filename, fid_distances, data_dir, tests_dir):

    df = pd.DataFrame()
    
    print("Processing...")
    
    with h5py.File(tests_dir+filename, "r") as f:
        
        #gammas = ['0.079477', '0.25133', '0.79477', '2.5133', '7.9477', '25.133', '79.477', '251.33']
        gammas = f.keys()
        
        for gamma in gammas:
            
            print(gamma)
            
            g = gamma[6:]
            
            init_states = f[gamma].keys()
            
            for state in init_states:
                
                F_lme_exact = f[gamma][state]["F_lme_exact"][()]
                F_sidexact_exact = f[gamma][state]["F_sidexact_exact"][()]
                F_sidlme_lme = f[gamma][state]["F_sidlme_lme"][()]                 
                
                InF_lme_exact = abs(1 - F_lme_exact)
                InF_sidexact_exact = abs(1 - F_sidexact_exact)
                InF_sidlme_lme = abs(1 - F_sidlme_lme)
                
                InF_lme_exact = [nf if nf>1e-15 else 1e-15 for nf in InF_lme_exact]
                InF_sidexact_exact = [nf if nf>1e-15 else 1e-15 for nf in InF_sidexact_exact]
                InF_sidlme_lme = [nf if nf>1e-15 else 1e-15 for nf in InF_sidlme_lme]
                
                
                
                ser_len = len(F_lme_exact)

                gamma_column = [g] * ser_len
                state_column = [state[7:]] * ser_len

                fid_dist_column = [fid_distances[int(state[7:])-1]] * ser_len      

                f_name = data_dir + "/" + state + "_2CUT_data.h5"
                #f_name = data_dir + "\\" + state + "_2CUT_data.h5"
                t, dt = extract_time(f_name, g)
                time_column = t
                gamma_time_column = np.array(t)*float(g) 

                d_ser = {'Gamma': gamma_column,
                         'State': state_column,
                         'Time': time_column, 
                         'gt': gamma_time_column,
                         'F_lme_exact': F_lme_exact,
                         'InF_lme_exact': InF_lme_exact,
                         'F_sidexact_exact': F_sidexact_exact,
                         'InF_sidexact_exact' : InF_sidexact_exact,
                         'F_sidlme_lme' : F_sidlme_lme,
                         'InF_sidlme_lme' : InF_sidlme_lme,
                         'Distance': fid_dist_column}

                df_ser = pd.DataFrame(data = d_ser)   
                df = pd.concat([df, df_ser])
    
    print(" done!")
    
    return df

In [5]:
#filename = "POP_fit_testD20_fix-Phi-up_2023-Aug-01_at_20-42.h5"
#filename = "POP_fit_trnD20_tstB4_fix-Phi-up_2023-Aug-02_at_13-24.h5"
#filename = "POP_fit_trn20_tst3__fix-Phi-up_2023-Aug-03_at_23-19.h5"

#filename = "POP_fit_trn4_gap_tst20_fix-Phi-up_2023-Aug-04_at_12-22.h5"

#filename = "POP_Rm_trn4_gap_tst20_fix-Phi-up_2023-Aug-04_at_17-07.h5"

#filename = "POP_Rm6_trn4_gap_tst20_fix-Phi-up_2023-Aug-04_at_18-40.h5"

#filename = "POP_Rm_523_trn4_gap_tst20_fix-Phi-up_2023-Aug-05_at_00-51.h5"

#filename = "POP_Rm_523_trn4_gap_tst20_fix-Phi-up_2023-Aug-06_at_20-29.h5"

#filename = "POP_RmLME_53354_trn4_gap_tst20_fix-Phi-up_2023-Aug-06_at_22-08.h5"

#filename = "POP_RmLME_53354_trn4_gap_tst20_fix-Phi-up_2023-Aug-06_at_22-58.h5"

#filename = "POP_RmLME_trn4_gap_tst20_fix-Phi-up_2023-Aug-07_at_15-44.h5"

#filename = "POP_NoRmLME_trn4_gap_tst20_fix-Phi-up_2023-Aug-07_at_16-29.h5"

filename = "POP_NoRmT0LME_trn4_gap_tst20_T0_fix-Phi-up_2023-Aug-10_at_12-14.h5"

In [6]:
df = read_as_dataframe_fits(filename, fid_dists, data_dir, tests_dir)

Processing...
gamma_0.079477
gamma_0.25133
gamma_0.79477
gamma_2.5133
gamma_25.133
gamma_251.33
gamma_7.9477
gamma_79.477
 done!


In [7]:
df

,Gamma,State,Time,gt,F_lme_exact,InF_lme_exact,F_sidexact_exact,InF_sidexact_exact,F_sidlme_lme,InF_sidlme_lme,Distance
0,0.079477,1,0.06000,0.004769,1.000000,6.039613e-14,1.000000,6.039613e-14,1.0,6.039613e-14,0.341925
1,0.079477,1,0.08000,0.006358,0.999997,2.694844e-06,0.999992,8.247467e-06,1.0,2.794928e-09,0.341925
2,0.079477,1,0.10000,0.007948,0.999982,1.777234e-05,0.999957,4.347965e-05,1.0,1.116189e-08,0.341925
3,0.079477,1,0.12000,0.009537,0.999972,2.793938e-05,0.999924,7.649190e-05,1.0,2.507431e-08,0.341925
4,0.079477,1,0.14000,0.011127,0.999986,1.365625e-05,0.999933,6.712687e-05,1.0,4.450590e-08,0.341925
...,...,...,...,...,...,...,...,...,...,...,...
751,79.477,9,0.23800,18.915526,0.788154,2.118457e-01,0.999316,6.835782e-04,1.0,1.000000e-15,0.277180
752,79.477,9,0.23825,18.935395,0.788127,2.118732e-01,0.999315,6.850773e-04,1.0,1.998401e-15,0.277180
753,79.477,9,0.23850,18.955264,0.788101,2.118987e-01,0.999313,6.865011e-04,1.0,1.000000e-15,0.277180
754,79.477,9,0.23875,18.975134,0.788078,2.119222e-01,0.999312,6.878479e-04,1.0,1.000000e-15,0.277180


In [8]:
os.chdir("/home/zah/POP23/TESTS/")

df.to_pickle("df_"+filename+".pkl")

In [9]:
"df_"+filename+".pkl"

'df_POP_NoRmT0LME_trn4_gap_tst20_T0_fix-Phi-up_2023-Aug-10_at_12-14.h5.pkl'